In [40]:
#Collective notebook
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [41]:
data = pd.read_csv("red_wine.csv")

In [42]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [43]:
# Predicting model (hea mudel + shufflemine)
# andmed: fixed a, volatile a, citric a, residual sugar, pH
# andmed: chlorides, free sulf, total sulf, density, sulphates

In [44]:
#Shuffling learning data
pred_data = data.copy()
df_shuf = pred_data.sample(frac=1).reset_index(drop=True)

In [45]:
train, test = train_test_split(df_shuf, test_size=0.2)
test1 = test.drop(columns="quality")
test.reset_index(inplace = True)

In [46]:
#Random Forrest:
X_train = train.drop(columns = "quality")
Y_train = train.quality
RF_balanced = RandomForestClassifier(n_estimators = 200, random_state = 0, min_samples_leaf = 3, min_samples_split = 3, max_depth = 31, class_weight="balanced_subsample" ).fit(X_train, Y_train)

In [47]:
arr = RF_balanced.predict_proba(test1)

In [48]:
#How many guesses were exactly correct with this model:
correct = 0
for i in range (len(arr)):
    predproba = 0
    pred = 0
    for k in range (len(arr[i])):
        if arr[i][k] > predproba:
            predproba = arr[i][k]
            pred = k + 3
    actual = test.iloc[i, 12]
    if pred == actual:
        correct += 1

In [49]:
print(correct/len(arr)*100)

69.6875


In [110]:
#KNN:
model = KNeighborsClassifier(n_neighbors = 1).fit(X_train, Y_train)
KNN_pred = model.predict(test1)

In [111]:
def KNN_results(arr, data):
    correct = 0
    for i in range(len(arr)):
        if(arr[i] == data.iloc[i, 12]):
            correct += 1
    return correct/len(arr)*100

In [112]:
KNN_results(KNN_pred, test)

49.0625

In [113]:
#Prediction with Good or Bad - Good quality > 5 

In [114]:
datapred = data.copy()
good_bad = [(datapred['quality'] < 6), (datapred['quality'] >= 6)]
datapred['quality_good'] = np.select(good_bad,['0','1'])
datapred = datapred.drop(columns="quality")
pred_shuf = datapred.sample(frac=1).reset_index(drop=True)
#pred_shuf = pred_shuf.drop(columns = "index")
X = pred_shuf.drop(['quality_good'], axis = 1)
y = pred_shuf.quality_good
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
#X = datapred.drop(['quality','quality_good'], axis = 1)
#y = datapred.quality_good

In [121]:
#Random Forrest:
from sklearn.metrics import accuracy_score
RF_good_bad = RandomForestClassifier(n_estimators = 400, min_samples_split = 2, min_samples_leaf = 1, max_features = "sqrt", max_depth = None, bootstrap = False).fit(X_train, y_train)
acc = accuracy_score(y_test, RF_good_bad.predict(X_test))
model = KNeighborsClassifier(n_neighbors = 5).fit(X_train, Y_train)
acc2 = accuracy_score(y_test, model.predict(X_test))
acc2

0.5025

In [105]:
if a:
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.ensemble import RandomForestRegressor
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
    print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [109]:
if a: 
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X_train, y_train)